# This is my first plotting script for CxAODReaderbbtt output

In [2]:
#!/usr/etc/env python
import ROOT

RuntimeError: property type mismatch or assignment not allowed

In [ ]:
# Select variable
def SelectVariable(list_of_hist, variable):
    """
    input: list of histogram, variable
    output: list contains all histo with variable
    """
    list_return = []
    for _hist in list_of_hist:

        _a = _hist.split('_')
        # print(_a)
        if _a[-1:][0] == variable:
            list_return.append(_hist)

    return list_return


def splitRegion(list_of_name):
    """
    input :such as [a,b,c,d,e,f]
    output: b_c_d_e
    """

    return list_of_name[1]+'_'+list_of_name[2]+'_'+list_of_name[3]+'_'+list_of_name[4]

# Get All histograms name in Preselection


def GetAllKeysInPreselection(tPre):
    """
     input: TPreselection
     return: All TH1F in TPreselection as a list

    """
    olist = []
    for key in tPre.GetListOfKeys():
        if key.GetName() == 'Systematics':
            continue
        olist.append(key.GetName())
    return olist

In [ ]:
# path for Hist-OUTPUT
dirHistOUTPUT = '/home/xuzifeng/HistxAOD/submitDir/hist-OUTPUT.root'
dir_save_figure='/home/xuzifeng/HistxAOD/submitDir/figure/'

# Select histograms we need!
region_tag = '2tag2pjet_0ptv_LL_OS'

# Get Preselection
file = ROOT.TFile(dirHistOUTPUT)
tPreselection = file.Get('Preselection')

In [ ]:
# Get all keys of histograms and save them in a list!

# Get All Keys!
ALLHIST=GetAllKeysInPreselection(tPreselection)
print(ALLHIST)

In [ ]:
# Get All Keys and sve them in table
dic_hist = {'Process': [], 'Region': [], 'Variable': []}

In [ ]:
# Just GetKeys and save them in a dictionary

for h_name in ALLHIST:
    h_name = h_name.split('_')
    process_name = h_name[0]
    region_name = splitRegion(h_name)
    variable_name = h_name[-1]

    if process_name not in dic_hist['Process']:
        dic_hist['Process'].append(process_name)
        # print(process_name)
        "config.addbackgroundSample({}, {}, k);".format(process_name, process_name)
    if region_name not in dic_hist['Region']:
        dic_hist['Region'].append(region_name)
    if variable_name not in dic_hist['Variable']:
        dic_hist['Variable'].append(variable_name)
        # print(variable_name)
# print(dic_hist)

In [ ]:
# Now stack all histogram needed and draw

# Remember check if the histogram we generate in the ALLHIST
# Generate our histogramlist
data_list = []
fake_list = []
process_list = []

In [ ]:
# Fill process_list
for _process in dic_hist['Process']:
    if _process == 'data':
        continue
    if _process == 'Fake':
        continue
    for _region in dic_hist['Region']:
        if _region == region_tag:
            for _variable in dic_hist['Variable']:
                a = _process + '_'+_region+'_'+_variable
                if a in ALLHIST:
                    process_list.append(a)
print(process_list)

In [ ]:
# Fill fake_list
for _process in dic_hist['Process']:
    if not(_process == 'Fake'):
        continue
    for _region in dic_hist['Region']:
        if _region == region_tag:
            for _variable in dic_hist['Variable']:
                a = _process + '_'+_region+'_'+_variable
                if a in ALLHIST:
                    fake_list.append(a)
print(fake_list)

In [ ]:
# Fill data_list
for _process in dic_hist['Process']:
    if not(_process == 'data'):
        continue
    for _region in dic_hist['Region']:
        if _region == region_tag:
            for _variable in dic_hist['Variable']:
                a = _process + '_'+_region+'_'+_variable
                if a in ALLHIST:
                    data_list.append(a)
print(data_list)

In [ ]:
dic_THStack={}
dic_TCanvas={}
for _variable in dic_hist['Variable']:
    
    # Create Canvas and THStacks!
    dic_TCanvas[_variable]=ROOT.TCanvas()
    dic_THStack[_variable]=ROOT.THStack(_variable, _variable)
print(dic_THStack)
print(dic_TCanvas)

In [ ]:
# if(debug==1):(process_list)
# if(debug==1):(SelectVariable(process_list, 'mBB'))

for _variable in dic_hist['Variable']:

    list_hist_Prediction = []
    list_hist_data = []
    if _variable == 'dEtaVBF':
        continue
    if _variable == 'mJJVBF':
        continue
    # divide TCanvas
    dic_TCanvas[_variable].Divide(1, 2)
    # Collecting histogram with differenct variable
    list_hist_Data = SelectVariable(data_list, _variable)
    hist_process = SelectVariable(process_list, _variable)
    hist_fake = SelectVariable(fake_list, _variable)
    hs_v = ROOT.THStack(_variable, 'test stacked histograms')

    """
    The Renomalize is wrong right now, it is needed to fix it!!!!
    Now it is fixed!
    """
    # This part Draw the upper pad of the figure
    norm_process_fake = 0
    norm_data = 0
    for _hist in hist_process:
        _histogram = tPreselection.Get(_hist)
        if _histogram.Integral() == 0:
            continue
            norm_process_fake = norm_process_fake+_histogram.Integral()
    for _hist in hist_fake:
        _histogram = tPreselection.Get(_hist)
        if _histogram.Integral() == 0:
            continue
        norm_process_fake = norm_process_fake+_histogram.Integral()
    for _hist in hist_fake:
        _histogram = tPreselection.Get(_hist)
        if _histogram.Integral() == 0:
            continue
        norm_process_fake = norm_process_fake+_histogram.Integral()
    for _hist in list_hist_Data:
        if _histogram.Integral() == 0:
            continue
        _histogram = tPreselection.Get(_hist)
        norm_data = norm_data+_histogram.Integral()

    for _hist in hist_process:
        _histogram = tPreselection.Get(_hist)
        if _histogram.Integral() == 0:
            continue
        _histogram.Scale(norm_process_fake/norm_data)
        dic_THStack[_variable].Add(_histogram)
        list_hist_Prediction.append(_histogram)
    for _hist in hist_fake:
        _histogram = tPreselection.Get(_hist)
        if _histogram.Integral() == 0:
            continue
        _histogram.Scale(norm_process_fake/norm_data)
        dic_THStack[_variable].Add(_histogram)
        list_hist_Prediction.append(_histogram)

    dic_TCanvas[_variable].cd(1)
    dic_THStack[_variable].Draw('HIST')
    ROOT.gPad.Update()
    # Draw data (only one)
    for _hist in list_hist_Data:
        if _histogram.Integral() == 0:
            continue
        _histogram = tPreselection.Get(_hist)
        ROOT.gPad.Update()
        _histogram.Draw('SAME')
        list_hist_data.append(_histogram)
    _Xaxis = dic_THStack[_variable].GetXaxis()
    _Yaxis = dic_THStack[_variable].GetYaxis()
    _Xaxis.SetTitle(_variable)
    _Yaxis.SetTitle('Events')

    # Now draw bottom panel

    dic_TCanvas[_variable].cd(2)
    predict=list_hist_Prediction[0]
    data=list_hist_data[0]
    for hist in list_hist_Prediction[1:]:
        predict=predict+hist
    for hist in list_hist_data[1:]:
        data=data+hist
    hist_ratio=data.Clone()
    hist_ratio.Divide(predict)
    hist_ratio.Draw()
    ROOT.gPad.Update()

    dic_TCanvas[_variable].Update()
    dic_TCanvas[_variable].SaveAs(_variable+'.pdf')